### Testing Loading of Full Human GEM model in Compass


In [1]:
from Compass.compass import models, compass, utils
import importlib
import pandas as pd
import os
import cplex

In [2]:
# Making changes in the local git repo - reload the .py file 
importlib.reload(models)
importlib.reload(models.importSBML3)
importlib.reload(models.geneSymbols)
importlib.reload(models.importCommon)

<module 'Compass.compass.models.importCommon' from '/home/dzhang/Compass_Integration/Compass/compass/models/importCommon.py'>

In [ ]:
model = models.load_metabolic_model("HumanGEM")

In [8]:
models.geneSymbols.resolve_genes(model)

In [1]:
import os, json
RESOURCE_DIR = "/home/dzhang/Compass_Integration/Compass/compass/Resources"

hgnc_file_PATH = os.path.join(RESOURCE_DIR, "Genes", "new_HGNC.json")

hgnc_file = open(hgnc_file_PATH)

data_dict = json.load(hgnc_file)

genes = data_dict 

In [5]:
import gzip
# Import list of genes with respective IDs
hgnc_file = os.path.join(
    RESOURCE_DIR, "Genes", "HGNC.json.gz"
)

data_dict = json.loads(gzip.open(hgnc_file).read().decode('utf-8'))

In [3]:
genes = data_dict['response']['docs']

In [34]:
type(genes)

list

In [17]:
# Rename the genes from ENSG ID to symbol
def update_association(assoc):
    if assoc.type == "gene":
        gene = assoc.gene
        if gene.name in gene_dict:
            gene_info = gene_dict[gene.name]
            gene.name = gene_info['symbol'].upper()
            if 'alias_symbol' in gene_info:
                gene.alt_symbols = [
                    x.upper() for x in gene_info['alias_symbol']
                ]
    else:
        for child in assoc.children:
            update_association(child)

In [18]:
for reaction in model.reactions.values():
    if reaction.gene_associations is not None:
        update_association(reaction.gene_associations)

In [69]:
model.reactions['MAR03905'].list_genes()

['ADH4',
 'ADH1B',
 'ADH-4',
 'ADH-5',
 'ADH7',
 'ADHX',
 'ADH-2',
 'ADH5',
 'ZADH2',
 'ADH-3',
 'ADHFE1',
 'MGC45594',
 'ADH1C',
 'GSNOR',
 'ADH1A',
 'FLJ32430',
 'ADH6']

Curious to see what else defines the Human-GEM SBML model 

In [118]:
sbml_document = models.load_metabolic_model("HumanGEM")
sbml_model = sbml_document.model

In [122]:
xml_params = {}
for pp in sbml_model.getListOfParameters():
    key = pp.getId()
    val = pp.getValue()
    xml_params[key] = val

In [149]:
import libsbml
for rxn in sbml_model.getListOfReactions():
    fbcrr = rxn.getPlugin('fbc')
    gpa = fbcrr.getGeneProductAssociation()
    print(gpa.getAssociation().getListOfAssociations()[8].getGeneProduct())

ENSG00000248144


AttributeError: 'GeneProductRef' object has no attribute 'getListOfAssociations'

In [4]:
len(model.reactions)

13073

In [38]:
list(model.reactions.items())[0][1].gene_associations.children[0].gene.alt_symbols

[]

In [31]:
total = 0
for k,v in model.reactions.items():
    r_id, rxn = (k, v)
    # Check if the gene association is empty 
    if rxn.gene_associations is None: 
        pass 
    elif rxn.gene_associations.list_genes().pop() == '':
        total += 1
total
#     if rxn.gene_associations is None: 
#         pass 
#     elif rxn.gene_associations.list_genes().pop() != '': 
#         print(rxn.gene_associations.list_genes())

8065

In [73]:
import libsbml 

In [74]:
sbmlDocument = libsbml.readSBMLFromFile("Human-GEM.xml")

In [75]:
level = sbmlDocument.getLevel()
level

3

In [76]:
xml_model = sbmlDocument.model

In [77]:
xml_params = {}
for pp in xml_model.getListOfParameters():
    key = pp.getId()
    val = pp.getValue()
    xml_params[key] = val

# Add reactions
for rr in xml_model.getListOfReactions():
    reaction = reaction_from_xml(rr, xml_params)
    modelx.reactions[reaction.id] = reaction

# Add compartments
for cp in xml_model.getListOfCompartments():
    compartment = compartment_from_xml(xml_node=cp)
    modelx.compartments[compartment.id] = compartment

# Add species
for met in xml_model.getListOfSpecies():
    species = species_from_xml(xml_node=met)
    modelx.species[species.id] = species

NameError: name 'reaction_from_xml' is not defined

In [27]:
xml_params = {}
for pp in xml_model.getListOfParameters():
    key = pp.getId()
    val = pp.getValue()
    xml_params[key] = val

xml_params

{'FB1N1000': -1000.0, 'FB2N0': 0.0, 'FB3N1000': 1000.0}

In [80]:
for xml_node in xml_model.getListOfReactions(): 
    ffbcrr = xml_node.getPlugin('fbc')
    gpa = ffbcrr.getGeneProductAssociation()
    if (gpa is not None): 
        print(gpa.getAssociation().getGeneProduct())

AttributeError: 'FbcOr' object has no attribute 'getGeneProduct'

In [79]:
for cp in xml_model.getListOfSpecies():
    print(cp.getPlugin('fbc').getChemicalFormula())
    print(cp.getPlugin('fbc') \
        .getChemicalFormula())

C10H16O
C10H16O
C10H16O
C10H16O
C10H16
C10H16
C10H16
C10H16
C17H31O2
C17H31O2
C17H31O2
C17H31O2
C17H31O2
C17H31O2
C17H31O2
C17H31O2
C38H62N7O17P3S
C38H62N7O17P3S
C38H62N7O17P3S
C38H62N7O17P3S
C38H62N7O17P3S
C38H62N7O17P3S
C20H31O4
C20H31O4
C43H72N7O17P3S
C43H72N7O17P3S
C43H72N7O17P3S
C43H72N7O17P3S
C43H72N7O17P3S
C43H72N7O17P3S
C41H68N7O17P3S
C41H68N7O17P3S
C41H68N7O17P3S
C41H68N7O17P3S
C41H68N7O17P3S
C41H68N7O17P3S
C41H68N7O17P3S
C41H68N7O17P3S
C20H35O2
C20H35O2
C20H35O2
C20H35O2
C20H35O2
C20H35O2
C20H35O2
C20H35O2
C41H66N7O17P3S
C41H66N7O17P3S
C41H66N7O17P3S
C41H66N7O17P3S
C41H66N7O17P3S
C41H66N7O17P3S
C20H33O2
C20H33O2
C20H33O2
C20H33O2
C20H33O2
C20H33O2
C20H33O2
C20H33O2
C27H47NO4
C27H47NO4
C27H47NO4
C27H47NO4
C27H47NO4
C27H47NO4
C41H64N7O17P3S
C41H64N7O17P3S
C41H64N7O17P3S
C41H64N7O17P3S
C41H64N7O17P3S
C41H64N7O17P3S
C20H31O5
C20H31O5
C19H25NO7S
C19H25NO7S
C40H56N8O22P3S2
C40H56N8O22P3S2
C43H72N7O17P3S
C43H72N7O17P3S
C43H72N7O17P3S
C43H72N7O17P3S
C43H72N7O17P3S
C43H72N7O17P3S
C43H

In [90]:
model = models.importSBML3.load("HumanGEM", xml_model)

In [109]:
# Load RECON2.2 model 
rmodel = models.load_metabolic_model("RECON2.2")

In [111]:
len(rmodel.reactions)

7785

In [132]:
# Checking for number of reactions with id/name 
total = 0
for key, value in model.reactions.items():
    name = value.name
    if name != "":
        total += 1
print(total)

6734


In [135]:
# Checking for gene name in gene association objects 
total = 0 
for k,v in rmodel.reactions.items():
    r_id, rxn = (k, v)
    # Check if the gene association is empty 
    if rxn.gene_associations is None: 
        total += 1
    else:
#         print(rxn.gene_associations.list_genes())
        total += 1
total

3043

In [84]:
for x in ffbcrr.getGeneProductAssociation().getAssociation().getListOfAssociations():
    gp_str = x.getGeneProduct()
    print(gp_str)
    fbmodel = x.getSBMLDocument().model.getPlugin('fbc')
    print(fbmodel)
    gp = fbmodel.getGeneProduct(gp_str)
    print(gp.getId())
    print(gp.getName())

ENSG00000147576
<FbcModelPlugin>
ENSG00000147576

ENSG00000172955
<FbcModelPlugin>
ENSG00000172955

ENSG00000180011
<FbcModelPlugin>
ENSG00000180011

ENSG00000187758
<FbcModelPlugin>
ENSG00000187758

ENSG00000196344
<FbcModelPlugin>
ENSG00000196344

ENSG00000196616
<FbcModelPlugin>
ENSG00000196616

ENSG00000197894
<FbcModelPlugin>
ENSG00000197894

ENSG00000198099
<FbcModelPlugin>
ENSG00000198099

ENSG00000248144
<FbcModelPlugin>
ENSG00000248144



### Creation of new_hgnc.json

In [9]:
# List of 13 HGNC IDs from Ensembl found from the Ensembl site (https://www.ensembl.org/index.html) 

ensembl_to_hgnc = {}
ensembl_to_hgnc['ENSG00000288702'] = 'HGNC:12535'
ensembl_to_hgnc['ENSG00000241635'] = 'HGNC:12530'
ensembl_to_hgnc['ENSG00000274252'] = 'HGNC:33426'
ensembl_to_hgnc['ENSG00000167165'] = 'HGNC:12538'
ensembl_to_hgnc['ENSG00000242366'] = 'HGNC:12540'
ensembl_to_hgnc['ENSG00000244474'] = 'HGNC:12536'
ensembl_to_hgnc['ENSG00000229894'] = 'HGNC:4292'
ensembl_to_hgnc['ENSG00000281500'] = 'HGNC:4061'
ensembl_to_hgnc['ENSG00000180389'] = 'HGNC:34026'
ensembl_to_hgnc['ENSG00000288705'] = 'HGNC:12537'
ensembl_to_hgnc['ENSG00000242515'] = 'HGNC:12531'
ensembl_to_hgnc['ENSG00000244122'] = 'HGNC:12539'
ensembl_to_hgnc['ENSG00000241119'] = 'HGNC:12541'

In [1]:
### Gene names are currently the Ensembl ids (i.e. equal to their gene ids) 
import json, gzip

hgnc_file = './Compass/compass/Resources/Genes/HGNC.json.gz'

data_dict = json.loads(gzip.open(hgnc_file).read().decode('utf-8'))
genes = data_dict['response']['docs']

In [4]:
key_field = 'hgnc_id'

In [5]:
gene_dict = {}
for gene in genes: 
    try:
        gene_dict[gene[key_field]] = gene
    except:
        pass 

In [10]:
# Check if the HGNC IDs exist in gene_dict already
for (ens, hgnc) in ensembl_to_hgnc.items():
    try: 
        print(hgnc)
        print(gene_dict[hgnc])
    except:
        print(ens, hgnc)

HGNC:12535
ENSG00000288702 HGNC:12535
HGNC:12530
ENSG00000241635 HGNC:12530
HGNC:33426
ENSG00000274252 HGNC:33426
HGNC:12538
ENSG00000167165 HGNC:12538
HGNC:12540
ENSG00000242366 HGNC:12540
HGNC:12536
ENSG00000244474 HGNC:12536
HGNC:4292
ENSG00000229894 HGNC:4292
HGNC:4061
{'gene_family': ['Solute carriers'], 'date_approved_reserved': '1998-07-15', 'vega_id': 'OTTHUMG00000166352', 'locus_group': 'protein-coding gene', 'status': 'Approved', 'alias_symbol': ['GSD1b', 'GSD1c', 'GSD1d'], '_version_': 1583018086391349251, 'uuid': 'd0d897db-b6fa-4e62-b6f3-a8692ef41454', 'prev_name': ['glucose-6-phosphatase, transport (glucose-6-phosphate) protein 1', 'solute carrier family 37 (glucose-6-phosphate transporter), member 4'], 'lsdb': ['LRG_187|http://www.lrg-sequence.org/LRG/LRG_187'], 'refseq_accession': ['NM_001467'], 'locus_type': 'gene with protein product', 'iuphar': 'objectId:1168', 'gene_family_id': [752], 'cosmic': 'SLC37A4', 'hgnc_id': 'HGNC:4061', 'rgd_id': ['RGD:62066'], 'ensembl_gene

In [12]:
# Identify the gene ids with ensembl ids and the indicies of the genes without 
names = []
unique_names = set()

empty_genes = []
empty_indicies = [] 
for i, gene in enumerate(genes): 
    try:
        names.append(gene['ensembl_gene_id'])
        unique_names.add(gene['ensembl_gene_id'])
    except:
        empty_genes.append(gene['hgnc_id'])
        empty_indicies.append(i)
        
empty_genes

['HGNC:53619',
 'HGNC:32925',
 'HGNC:942',
 'HGNC:15728',
 'HGNC:15730',
 'HGNC:53646',
 'HGNC:24799',
 'HGNC:53611',
 'HGNC:31957',
 'HGNC:31958',
 'HGNC:31959',
 'HGNC:4105',
 'HGNC:5477',
 'HGNC:5715',
 'HGNC:5853',
 'HGNC:6225',
 'HGNC:29111',
 'HGNC:13384',
 'HGNC:8662',
 'HGNC:8679',
 'HGNC:8695',
 'HGNC:38749',
 'HGNC:53639',
 'HGNC:12027',
 'HGNC:12155',
 'HGNC:12252',
 'HGNC:12271',
 'HGNC:12529',
 'HGNC:53444',
 'HGNC:30679',
 'HGNC:21508',
 'HGNC:18372']

In [17]:
RESOURCE_DIR = "/home/dzhang/Compass_Integration/Compass/compass/Resources"
hgnc_file = os.path.join(RESOURCE_DIR, "Genes", "new_HGNC.json")

f = open(hgnc_file)

data_dict = json.load(f)

In [142]:
hgnc_json_PATH = './Compass/compass/Resources/Genes/HGNC.json'
f = open(hgnc_json_PATH) 

data = json.load(f)
hgnc_genes = data['response']['docs']

# Test diffs bw hgnc_genes json file from Compass repo and gene from json.gz file - no difference

for i in range(len(hgnc_genes)): 
    if not hgnc_genes[i] == genes[i]: 
        print(hgnc_genes[i])
        print(genes[i])

In [143]:
import httplib2 as http
import json
import requests 

def hgnc_id_to_gene_info(hgnc_id): 
    # Testing of the HGNC API to retrieve HGNC symbol from ID
    

    headers = {
      'Accept': 'application/json',
    }

    url = f'https://rest.genenames.org/fetch/hgnc_id/{hgnc_id}'
    method = 'GET'
    body = ''

    h = http.Http()

    response, content = h.request(
      url,
      method,
      body,
      headers)

    if response['status'] == '200': 
        data = json.loads(content)
        return data['response']['docs']
    else: 
        return 1

In [144]:
# Update the HGNC JSON 

## Update HGNC 4061 to new associated Ensembl
gene_dict['HGNC:4061']['ensembl_gene_id'] = 'ENSG00000281500'
del ensembl_to_hgnc['ENSG00000281500']

## Update remaining 12 w/ the newly associated HGNC ID and HGNC 'symbol' attribute 

# Create initial gene_info dict to append into the new key-value field with HGNC ID 
for (ens, hgnc) in ensembl_to_hgnc.items():
    hgnc_id = hgnc[5:]
    gene_info = hgnc_id_to_gene_info(hgnc_id)[0]
    if gene_info == 1: 
        print("Error, response code was not 200")
        print(ens, hgnc)
    else: 
        gene_info['ensembl_gene_id'] = ens 
        gene_dict[hgnc] = gene_info

In [145]:
# Generating JSON file from updated dict 
import json 
with open('new_HGNC.json', 'w') as fp:
    json.dump(gene_dict, fp)

In [146]:
len(gene_dict)

19128

In [20]:
gene_dict

{'HGNC:5': {'gene_family': ['Immunoglobulin like domain containing'],
  'date_approved_reserved': '1989-06-30',
  'vega_id': 'OTTHUMG00000183507',
  'locus_group': 'protein-coding gene',
  'status': 'Approved',
  '_version_': 1583018081624522752,
  'uuid': '6314a37d-67f1-499a-bf8f-e9d03625cb8e',
  'merops': 'I43.950',
  'refseq_accession': ['NM_130786'],
  'locus_type': 'gene with protein product',
  'gene_family_id': [594],
  'cosmic': 'A1BG',
  'hgnc_id': 'HGNC:5',
  'rgd_id': ['RGD:69417'],
  'ensembl_gene_id': 'ENSG00000121410',
  'entrez_id': '1',
  'omim_id': ['138670'],
  'symbol': 'A1BG',
  'location': '19q13.43',
  'name': 'alpha-1-B glycoprotein',
  'date_modified': '2015-07-13',
  'mgd_id': ['MGI:2152878'],
  'ucsc_id': 'uc002qsd.5',
  'uniprot_ids': ['P04217'],
  'ccds_id': ['CCDS12976'],
  'pubmed_id': [2591067],
  'location_sortable': '19q13.43'},
 'HGNC:24086': {'gene_family': ['RNA binding motif containing'],
  'date_approved_reserved': '2007-11-23',
  'vega_id': 'OTTHU

In [46]:
# Total number of unique Ensembl IDs in Human GEM model 
unique_ids = set() 
for id_key, rxn in model.reactions.items():
    for gene in rxn.list_genes():
        unique_ids.add(gene)
len(unique_ids)

2897

In [21]:
# Check to see if all the Human GEM genes have corresponding Ensembl ids from HGNC file 
absent_genes = []
for e_id in unique_ids:
    if e_id not in names:
        absent_genes.append(e_id)

NameError: name 'unique_ids' is not defined

In [ ]:
set(absent_gene)

In [20]:
list(absent_gene_rxns)

NameError: name 'absent_gene_rxns' is not defined

In [339]:
# Identify the rxns associated with each absent gene
absent_gene_rxns = {gene : [] for gene in absent_genes}

for gene in absent_genes:
    for id_key, rxn in model.reactions.items():
        if gene in rxn.list_genes(): 
            absent_gene_rxns[gene].append((rxn.id, rxn.name))
            
absent_gene_rxns['ENSG00000241119']

[('MAR06798', ''),
 ('MAR06799', ''),
 ('MAR06800', ''),
 ('MAR06801', ''),
 ('MAR06828', ''),
 ('MAR06830', ''),
 ('MAR06836', ''),
 ('MAR06837', ''),
 ('MAR06842', ''),
 ('MAR06843', ''),
 ('MAR02461', ''),
 ('MAR02462', ''),
 ('MAR02463', ''),
 ('MAR02464', ''),
 ('MAR02467', ''),
 ('MAR02468', ''),
 ('MAR02469', ''),
 ('MAR02470', ''),
 ('MAR04766',
  'UDPglucuronate beta-D-glucuronosyltransferase(acceptor-unspecific)'),
 ('MAR07648',
  'UDPglucuronate beta-D-glucuronosyltransferase(acceptor-unspecific)'),
 ('MAR00026', ''),
 ('MAR06666', ''),
 ('MAR06667', ''),
 ('MAR06668', ''),
 ('MAR06669', ''),
 ('MAR06679', ''),
 ('MAR06680', ''),
 ('MAR06685', ''),
 ('MAR06686', ''),
 ('MAR06704', ''),
 ('MAR06705', ''),
 ('MAR06490', ''),
 ('MAR06495', ''),
 ('MAR07039', 'UDP-glucuronate beta-D-glucuronosyltransferase'),
 ('MAR07065', ''),
 ('MAR07066', ''),
 ('MAR09573', ''),
 ('MAR09574', ''),
 ('MAR02617', 'Glucuronosyltransferase'),
 ('MAR02627', 'Glucuronosyltransferase'),
 ('MAR02631'